In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import datetime
import pandas as pd
import numpy as np

In [2]:
# load trained model,scaler pickel ,onehot
model=load_model('model.h5')

# load encoder and scaler
with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender= pickle.load(file)
with open('one_hot_encoder.pkl', 'rb') as file:
    one_hot_encoder= pickle.load(file)
with open('scaler.pkl', 'rb') as file:
    scaler= pickle.load(file)


In [3]:
# Example Input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender':'Male',
    'Age': 40,
    'Tenure': 5,
    'Balance': 10000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [4]:
# one-Hot encoding for Geography
geo_encoded = one_hot_encoder.transform([[input_data['Geography']]]).toarray()
geo_encoded_df=pd.DataFrame(geo_encoded, columns=one_hot_encoder.get_feature_names_out(['Geography']))
geo_encoded_df


e:\Churn\myenv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [5]:
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,5,10000,2,1,1,50000


In [6]:
# encode categorical variable
input_df['Gender'] = input_df['Gender'].astype(str).str.strip()

In [7]:
# Check encoder classes and input values
print(label_encoder_gender.classes_)
print(input_df['Gender'].unique())

# Clean and encode
gender_mapping = {'Male': 1, 'Female': 0}
input_df['Gender'] = input_df['Gender'].map(gender_mapping)


[0 1]
['Male']


In [8]:
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,5,10000,2,1,1,50000


In [10]:
# concatenating the data
input_df = pd.concat([input_df.drop("Geography",axis=1), geo_encoded_df],axis= 1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,5,10000,2,1,1,50000,1.0,0.0,0.0


In [11]:
# scanling the input data
input_df_scaled = scaler.transform(input_df)
input_df_scaled

array([[-0.53598516,  0.91324755,  0.10479359, -0.00134472, -1.05836066,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [14]:
# predictions
prediction=model.predict(input_df_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step


array([[4.3269698e-05]], dtype=float32)

In [15]:
prediction_proba=prediction[0][0]
prediction_proba

np.float32(4.3269698e-05)

In [16]:
if prediction_proba > 0.5:
    print(f"Customer is likely to churn with a probability of {prediction_proba:.2f}")      
else:
    print(f"Customer is unlikely to churn with a probability of {1 - prediction_proba:.2f}")

Customer is unlikely to churn with a probability of 1.00
